# Pre-processing

In [ ]:
# importing libraries

import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Activation, Dropout
from keras import backend as K
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

In [ ]:
# retriving dataset

!gdown --id -q 1N_iNvJ8zXfCiSb5t3IqjH1JMDcHUEYnj
data = pd.read_csv("Iris.csv")

In [ ]:
# replacing output column

dummies = pd.get_dummies(data["Species"])
data["Species"] = dummies.values.tolist()

In [ ]:
# shuffling data

shuffled_data = data.sample(frac=1, random_state=1).reset_index(drop=True)
print(shuffled_data)

In [ ]:
# splitting input and output

X = shuffled_data.drop(["Id", "Species"], axis=1).values
Y = np.vstack(shuffled_data["Species"].tolist())

In [ ]:
# normalizing input

X = normalize(X, axis=0)

In [ ]:
# splitting training, validation and test sets

X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size = 0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size = 0.25)

# Model

In [ ]:
def create_model(hidden_layers, activation, optimizer, learning_rate):  
  model = Sequential()
  model.add(Dense(h1, activation = activation, input_dim = 4))
  for i in range(hidden_layers - 1):
    model.add(Dense(h2, activation = activation))
  model.add(Dense(3, activation = 'sigmoid'))
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  K.set_value(model.optimizer.learning_rate, learning_rate)
  return model

In [ ]:
def evaluate(model):
  _, accuracy = model.evaluate(X_train, Y_train, verbose=0)
  print('Training Accuracy: %.2f' % (accuracy*100))
  _, accuracy = model.evaluate(X_val, Y_val, verbose=0)
  print('Validation Accuracy: %.2f' % (accuracy*100))

In [ ]:
def test(model):
  _, accuracy = model.evaluate(X_test, Y_test, verbose=0)
  print('Test Accuracy: %.2f' % (accuracy*100))

# Evaluation

In [ ]:
# hyperparameters

h1 = 100
h2 = 100
epochs = 50
learning_rate = 0.1
activation = "relu"
optimizer = "adam"

In [ ]:
model1 = create_model(hidden_layers = 2, activation = activation, optimizer = optimizer, learning_rate = learning_rate)
history = model1.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = epochs, batch_size = X_train.shape[0], verbose=0)

In [ ]:
evaluate(model1)
test(model1)

In [ ]:
train_err = history.history["loss"]
val_err = history.history["val_loss"]

fig_err = go.Figure()
fig_err.add_trace(go.Scatter(x=list(range(len(train_err))), y=train_err, name="Training Error", mode='lines+markers'))
fig_err.add_trace(go.Scatter(x=list(range(len(val_err))), y=val_err, name="Validation Error", mode='lines+markers'))
fig_err.update_layout(title = f'Error vs Iterations',title_x=0.5, xaxis_title= "Iterations", yaxis_title="Error")

fig_err.show(renderer="svg")

In [ ]:
train_acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

fig_acc = go.Figure()
fig_acc.add_trace(go.Scatter(x=list(range(len(train_acc))), y=train_acc, name="Training Accuracy", mode='lines+markers'))
fig_acc.add_trace(go.Scatter(x=list(range(len(val_acc))), y=val_acc, name="Validation Accuracy", mode='lines+markers'))
fig_acc.update_layout(title = f'Accuracy vs Iterations',title_x=0.5, xaxis_title= "Iterations", yaxis_title="Accuracy")

fig_acc.show(renderer="svg")

# Experiments

In [ ]:
import time

## Different Number of Hidden Layers

In [ ]:
print("2 Hidden Layers")
model2 = create_model(hidden_layers = 2, activation = "relu", optimizer = "adam", learning_rate = 0.01)
model2.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
evaluate(model2)

print("\n4 Hidden Layers")
model3 = create_model(hidden_layers = 4, activation = "relu", optimizer = "adam", learning_rate = 0.01)
model3.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
evaluate(model3)

print("\n6 Hidden Layers")
model4 = create_model(hidden_layers = 6, activation = "relu", optimizer = "adam", learning_rate = 0.01)
model4.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
evaluate(model4)

## Different Activation Functions

In [ ]:
print("ReLU Activation")

model5 = create_model(hidden_layers = 6, activation = "relu", optimizer = "adam", learning_rate = 0.01)
t1 = time.perf_counter()
model5.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
t2 = time.perf_counter()
evaluate(model5)
print(f"Execution Time: {t2 - t1}")

print("\nSigmoid Activation")
model6 = create_model(hidden_layers = 6, activation = "sigmoid", optimizer = "adam", learning_rate = 0.01)
t1 = time.perf_counter()
model6.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
t2 = time.perf_counter()
evaluate(model6)
print(f"Execution Time: {t2 - t1}")

print("\nHyperbolic Tangent Activation")
model7 = create_model(hidden_layers = 6, activation = "tanh", optimizer = "adam", learning_rate = 0.01)
t1 = time.perf_counter()
model7.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
t2 = time.perf_counter()
evaluate(model7)
print(f"Execution Time: {t2 - t1}")

## Different Optimizers

In [ ]:
print("Adam Optimizer")
model8 = create_model(hidden_layers = 6, activation = "tanh", optimizer = "adam", learning_rate = 0.01)
t1 = time.perf_counter()
model8.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
t2 = time.perf_counter()
evaluate(model8)
print(f"Execution Time: {t2 - t1}")

print("\nAdagrad Optimizer")
model9 = create_model(hidden_layers = 6, activation = "tanh", optimizer = "adagrad", learning_rate = 0.01)
t1 = time.perf_counter()
model9.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
t2 = time.perf_counter()
evaluate(model9)
print(f"Execution Time: {t2 - t1}")

print("\nAdadelta Optimizer")
model10 = create_model(hidden_layers = 6, activation = "tanh", optimizer = "adadelta", learning_rate = 0.01)
t1 = time.perf_counter()
model10.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
t2 = time.perf_counter()
evaluate(model10)
print(f"Execution Time: {t2 - t1}")

## Different Learning Rates

In [ ]:
print("Learning Rate = 1")
model11 = create_model(hidden_layers = 6, activation = "tanh", optimizer = "adam", learning_rate = 1)
t1 = time.perf_counter()
model11.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
t2 = time.perf_counter()
evaluate(model11)
print(f"Execution Time: {t2 - t1}")

print("\nLearning Rate = 0.1")
model12 = create_model(hidden_layers = 6, activation = "tanh", optimizer = "adam", learning_rate = 0.1)
t1 = time.perf_counter()
model12.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
t2 = time.perf_counter()
evaluate(model12)
print(f"Execution Time: {t2 - t1}")

print("\nLearning Rate = 0.01")
model13 = create_model(hidden_layers = 6, activation = "tanh", optimizer = "adam", learning_rate = 0.01)
t1 = time.perf_counter()
model13.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 50, batch_size = X_train.shape[0], verbose=0)
t2 = time.perf_counter()
evaluate(model13)
print(f"Execution Time: {t2 - t1}")